In [20]:
from pathlib import Path

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import os
from tqdm import tqdm
import numpy as np
import torchvision.models as models

import torch.optim as optim

import torchvision.transforms as v1
from torchvision.io import read_image

plt.rcParams["savefig.bbox"] = 'tight'
torch.manual_seed(1)

import torch, torchvision
from torch.utils.data.dataloader import DataLoader
from sklearn.model_selection import train_test_split
import csv
import os, shutil
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import mlconfig
from importlib import import_module
from torch.utils.data import Dataset, DataLoader
import pandas as pd

def load_function(attr):
    module_, func = attr.rsplit('.', maxsplit=1)
    return getattr(import_module(module_), func)

#config = mlconfig.load('config_resnet18.yaml')
root_dir = 'C:\\Users\\Admin/'

In [2]:
!pip install mlconfig

In [22]:
with open(root_dir+'/'+'data_120.csv', newline='') as csvfile:
            s = []
            spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
            for row in list(spamreader)[1:]:
                row = str(row)
                s.append(row[2:len(row)-2])

In [38]:
folder_path = 'C:\\Users\\Admin/test_ready'

In [ ]:
shutil.rmtree(folder_path)

In [23]:
class MyDataset(Dataset):
    def __init__(self, root_dir, csv_file):
        s=[]
        with open(root_dir+'/'+csv_file, newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
            for row in list(spamreader)[1:]:
                row = str(row)
                s.append(row[2:len(row)-2])
            for i in range(len(s)):
                s[i] = s[i].split(',')
        self.sig_filenames = np.array(s)[:,0]
        self.root_dir = root_dir

    def __len__(self):
        return len(self.sig_filenames)

    def __getitem__(self, idx):
        data_file = np.asarray(np.load(self.root_dir + '/test_ready/' + self.sig_filenames[idx] + '.npy', 'r+'), dtype=np.float32)
        if 'drone' in self.sig_filenames[idx]:
            label = torch.tensor(1)
        if 'noise' in self.sig_filenames[idx]:
            label = torch.tensor(0)
        return data_file, label

In [33]:
dataset = MyDataset(root_dir, 'data_120.csv')

In [29]:
train_set, valid_set = torch.utils.data.random_split(dataset, [0.05, 0.95], generator=torch.Generator().manual_seed(42))
#batch_size = config.batch_size
train_dataloader = torch.utils.data.DataLoader(
  train_set,
  batch_size=1,
  shuffle=True,
  drop_last=True)
valid_dataloader = torch.utils.data.DataLoader(
  valid_set,
  batch_size=1,
  drop_last=True,
  shuffle=True)

In [34]:
dataloaders = {}
dataloaders['train'] = train_dataloader
dataloaders['val'] = valid_dataloader
dataset_sizes = {}
dataset_sizes['train'] = len(train_set)
dataset_sizes['val'] = len(valid_set)

In [35]:
model = models.resnet18(pretrained=False)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [39]:
list(model.parameters())

[Parameter containing:
 tensor([[[[ 2.2940e-02, -1.1197e-02,  5.0699e-03,  ..., -1.0984e-02,
             2.7693e-03, -1.2448e-02],
           [-3.5945e-02,  9.3781e-03, -6.0806e-03,  ...,  3.1114e-02,
            -1.0628e-01,  2.1248e-02],
           [-9.4673e-03, -2.5203e-02,  3.3479e-02,  ..., -6.3524e-03,
            -2.7172e-02, -2.4979e-03],
           ...,
           [ 7.0443e-03,  3.9991e-02,  1.4913e-03,  ..., -5.2721e-02,
            -2.7985e-02,  3.1030e-02],
           [ 1.8927e-02,  1.9720e-02, -7.7225e-03,  ...,  1.8559e-02,
            -4.2555e-02, -1.1063e-02],
           [ 7.8338e-03, -1.7682e-03,  9.9689e-03,  ..., -2.2083e-02,
             3.4740e-02,  9.9347e-03]],
 
          [[ 1.8000e-02, -6.1819e-03,  4.0725e-02,  ..., -2.0401e-02,
            -1.7554e-02,  3.9864e-02],
           [ 2.7610e-02,  5.5658e-02,  1.5066e-02,  ..., -2.6754e-02,
             1.5749e-02, -5.3730e-02],
           [ 2.3066e-03, -3.0355e-03,  2.1827e-02,  ..., -5.1959e-02,
            -2.1

In [36]:
torch.cuda.empty_cache()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

import numpy as np
import copy, timeit

if device != 'cpu':
    model = model.to(device)

val_loss = []
val_acc = []
train_loss = []
train_acc = []
epochs = 50

best_acc = 0
best_model = copy.deepcopy(model.state_dict())

start = timeit.default_timer()
for epoch in range(1, epochs+1):
    print(f"Epoch : {epoch}\n")
    dataloader = None

    for phase in ['train', 'val']:
        running_loss = 0.0
        running_corrects = 0

        for (img, label) in tqdm(dataloaders[phase]):
            img, label = img.to(device), label.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                output = model(img)
                _, pred = torch.max(output.data, 1)
                loss = criterion(output, label)
                if phase=='train' :
                    loss.backward()
                    optimizer.step()
            
            running_loss += loss.item() * img.size(0)
            running_corrects += torch.sum(pred == label.data)

        print(running_loss, dataset_sizes[phase], running_corrects)
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
                
        if phase=='train' :
            train_loss.append(epoch_loss)
            train_acc.append(epoch_acc)
        else :
            val_loss.append(epoch_loss)
            val_acc.append(epoch_acc)
            if val_acc[-1] > best_acc :
                best_acc = val_acc[-1]
                best_model = copy.deepcopy(model.state_dict())
    print()

end = timeit.default_timer()
print(f"Total time elapsed = {end - start} seconds")
name = 'resnet_18_new_new.pth'
torch.save(best_model, name)

cuda
Epoch : 1



100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00, 11.87it/s]


51.76817400753498 12 tensor(4, device='cuda:0')
train Loss: 4.3140 Acc: 0.3333


100%|████████████████████████████████████████████████████████████████████████████████| 227/227 [00:12<00:00, 18.13it/s]


226.13526538014412 227 tensor(111, device='cuda:0')
val Loss: 0.9962 Acc: 0.4890

Epoch : 2



...

11.264011378865689 227 tensor(227, device='cuda:0')
val Loss: 0.0496 Acc: 1.0000

Epoch : 50



100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 14.12it/s]


0.023889831034466624 12 tensor(12, device='cuda:0')
train Loss: 0.0020 Acc: 1.0000


100%|████████████████████████████████████████████████████████████████████████████████| 227/227 [00:12<00:00, 18.10it/s]


11.239408148219809 227 tensor(227, device='cuda:0')
val Loss: 0.0495 Acc: 1.0000

Total time elapsed = 650.1541792999997 seconds
